# WARNING

The preview for this file in Github is broken. **DO NOT copy and paste from the preview.** Read the README for further instructions.

# Extract, Transform

In [1]:
import requests
import pandas as pd
import numpy as np
import gc

willStore = False
lines = []

for line in requests.get('https://creatures-of-sonaria-official.fandom.com/wiki/Module:CreatureData/data').text.split('\n'):
    line = line.strip()
    if line == '' or '<meta' in line:
        willStore = False
    if willStore:
        lines.append(line)
    if 'p.creatureStats' in line:
        willStore = True

strings = ''.join(lines).replace('&#039;','').replace('&quot;','"').replace('&lt;','').replace('&gt;','').split('\t')[:-1]

columns = ['name']

for string in strings:
    if '}' in string:
        break
    if '{' not in string:
        columns.append(string[1:string.index(']')])

data = [[]]
index = 0

for string in strings:
    #precautionary data cleaning
    string = string.replace(').','),').replace('  ', ' ')

    if '[[' in string:
        string = string[:string.index('[[')] + string[string.rindex(']]')+2:]
    if '}' in string:
        index += 1
        data.append([])
    if '"' in string:
        temp = string[string.index('"')+1:string.rindex('"')].strip()
        if temp.endswith('min'):
            temp = temp[:temp.index('min')-1]
        if temp.isdecimal():
            temp = int(temp)
        elif temp.replace('.','').isdecimal():
            temp = float(temp)
        elif temp == 'N/A':
            temp = np.nan
        data[index].append(temp)
    else:
        data[index].append(string[string.index('/')-2:string.rindex(',')])

df = pd.DataFrame(data, columns=columns)

if all(df['name'] == df['common']):
    df.drop(columns='common', inplace=True)
df.drop(columns=[col for col in df.columns if df[col].nunique(dropna=False) == 1], inplace=True) #drops hungerDrain and thirstDrain

eWeightList = []
eDPSList = []
eHPList = []
for i in range(df.shape[0]):
    eWeight = 1
    eDPS = 1
    eHP = 1
    if isinstance(df.at[i, 'activated'], str):
        if 'Fortify' in df.at[i, 'activated']:
            eWeight += 0.05
        if 'Harden' in df.at[i, 'activated']:
            eWeight += 0.25
        if 'Adrenaline' in df.at[i, 'activated']:
            eDPS += 0.2
        if 'Unbridled Rage' in df.at[i, 'activated']:
            eDPS += 0.3
        if 'Wardens Rage' in df.at[i, 'activated']:
            eDPS += 5.625
        if 'Cause Fear' in df.at[i, 'activated']:
            eHP += 1
    if isinstance(df.at[i, 'passive'], str) and 'Berserk' in df.at[i, 'passive']:
        eDPS += 0.2
    eWeight *= df.at[i, 'weight']
    eWeightList.append(eWeight)
    eDPS *= df.at[i, 'damage'] * (1 + eWeight / 50000) / (2 * df.at[i, 'biteCooldown'])
    eDPSList.append(eDPS)
    eHP *= df.at[i, 'health'] / (1 + 50000 / eWeight)
    eHPList.append(eHP)
df['effectiveWeight'] = eWeightList
df['effectiveDPS'] = eDPSList
df['effectiveHP'] = eHPList
df['effectivePower'] = df['effectiveDPS'] * df['effectiveHP']

def flySprintMultiplier(fsm: float) -> float:
    if np.isnan(fsm):
        return 1
    return 1 + fsm
def ambush(a: float) -> float:
    if np.isnan(a):
        return 1
    return a
def agileSwimmer(passive) -> float:
    if isinstance(passive, str) and 'Agile Swimmer' in passive:
        return 1.75
    return 1
df['flySprintSpeed'] = df['flySpeed'] * df['flySprintMultiplier'].apply(flySprintMultiplier)
df['swimSprintSpeed'] = df['sprintSpeed'] * df['passive'].apply(agileSwimmer)
df['ambushSpeed'] = df['sprintSpeed'] * df['ambush'].apply(ambush)
df['swimAmbushSpeed'] = df['swimSprintSpeed'] * df['ambush'].apply(ambush)
df['effectiveHPRegen'] = df['effectiveHP'] * df['healthRegen']

def wardensRage(string):
    if isinstance(string, str):
        return string.replace('Wardens Rage', "Warden's Rage")
    return string
df['activated'] = df['activated'].apply(wardensRage)

def split(string) -> list[str]:
    if not isinstance(string, str):
        return []
    list1 = string.split(',')
    for string1 in list1:
        string1 = string1.strip()
    return list1
passives = df['passive'].apply(split)
activateds = df['activated'].apply(split)
tempDF = pd.merge(df['name'], passives, left_index=True, right_index=True)
passiveDF = tempDF.explode('passive', ignore_index=True)
tempDF = pd.merge(df['name'], activateds, left_index=True, right_index=True)
activatedDF = tempDF.explode('activated', ignore_index=True)

spreadsheetDF = df.copy()
df.drop(columns=['passive', 'activated'], inplace=True)

for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].str.strip()
for col in spreadsheetDF.columns:
    if spreadsheetDF[col].dtype == 'object':
        spreadsheetDF[col] = spreadsheetDF[col].str.strip()
for col in passiveDF.columns:
    passiveDF[col] = passiveDF[col].str.strip()
for col in activatedDF.columns:
    activatedDF[col] = activatedDF[col].str.strip()

del willStore, lines, strings, line, columns, data, index, temp, string, eWeightList, eDPSList, eHPList, eWeight, eDPS, eHP, i, flySprintMultiplier, ambush, agileSwimmer, wardensRage, split, passives, activateds, tempDF, col
gc.collect()

189

# Verify

In [3]:
#This code produces 6 different outputs: Suspicious flyers, unique passive abilities, unique activated abilities,
#miscategorized abilities, rows in spreadsheetDF with trailing commas in passive or activated,
#and number of missing values in each column.
pd.set_option("display.max_columns", 50)
pd.set_option("display.max_rows", 500)

print('\nSuspicious flyers:')
df.reset_index(drop=True, inplace=True)
for i in range(df.shape[0]):
    if not np.isnan(df.at[i, 'flySpeed']):
        if np.isnan(df.at[i, 'flySprintMultiplier']) or np.isnan(df.at[i, 'glideStaminaRegen']) or np.isnan(df.at[i, 'takeoffMultiplier']):
            print(df.loc[i, ['name', 'flySpeed', 'flySprintMultiplier', 'glideStaminaRegen', 'takeoffMultiplier']])

def fillnan(string) -> str:
    if isinstance(string, str):
        return string
    return ''
print('\nUnique passive abilities:')
print(np.sort(passiveDF['passive'].apply(fillnan).unique()))
print('\nUnique activated abilities:')
print(np.sort(activatedDF['activated'].apply(fillnan).unique()))
print('\nMiscategorized abilities:')
print(pd.merge(passiveDF, activatedDF, left_on='passive', right_on='activated')['passive'].apply(fillnan).unique())
print('\nTrailing commas:')
spreadsheetDF.reset_index(drop=True, inplace=True)
for i in range(spreadsheetDF.shape[0]):
    if isinstance(spreadsheetDF.at[i, 'passive'], str) and spreadsheetDF.at[i, 'passive'].strip().endswith(','):
        print(spreadsheetDF.loc[i, ['name', 'passive']])
    if isinstance(spreadsheetDF.at[i, 'activated'], str) and spreadsheetDF.at[i, 'activated'].strip().endswith(','):
        print(spreadsheetDF.loc[i, ['name', 'activated']])
print('\nNumber of missing values in each column:')
df.isnull().sum()


Suspicious flyers:
name                   Exotide
flySpeed                  48.0
flySprintMultiplier        NaN
glideStaminaRegen          1.0
takeoffMultiplier         10.0
Name: 93, dtype: object
name                   Umbraxi
flySpeed                  28.0
flySprintMultiplier        0.3
glideStaminaRegen          NaN
takeoffMultiplier         25.0
Name: 317, dtype: object

Unique passive abilities:
['Agile Swimmer' 'Berserk' 'Bleed Attack (-0.5)' 'Bleed Attack (-1)'
 'Bleed Attack (-1.5)' 'Bleed Attack (-2)' 'Bleed Attack (-3)'
 'Bleed Attack (0.5)' 'Bleed Attack (1)' 'Bleed Attack (1.5)'
 'Bleed Attack (10)' 'Bleed Attack (15)' 'Bleed Attack (2)'
 'Bleed Attack (2.5)' 'Bleed Attack (20)' 'Bleed Attack (3)'
 'Bleed Attack (3.5)' 'Bleed Attack (4)' 'Bleed Attack (4.5)'
 'Bleed Attack (5)' 'Bleed Attack (5.5)' 'Bleed Attack (6)'
 'Bleed Attack (7.5)' 'Bleed Attack (8)' 'Bleed Attack (9)'
 'Block Bleed (-0.10)' 'Block Bleed (-0.15)' 'Block Bleed (-0.25)'
 'Block Bleed (-0.30)' 'Block 

,0
name,0
ambush,194
appetite,0
beachSpeed,339
biteCooldown,0
bloodColor,0
bloodTexture,0
breath,251
type,0
damage,0


# Clean

Checklist:
* Some of the "Breath Resistance", "Block (Status)", "First Strike", "Toxic Trail", and "Life Leech" abilities display as 0.1 instead of 0.10, 0.2 instead of 0.20, and so on up to 1.0 instead of 1.00. ✅
* Pernirex, Shro, and Tsugae have "Egg Stealer" listed in passive abilities instead of in activated abilities. ✅

In [2]:
df.set_index('name', drop=False, inplace=True, verify_integrity=True)
spreadsheetDF.set_index('name', drop=False, inplace=True)

#Egg Stealer fix
for name in ('Pernirex', 'Shro', 'Tsugae'):
    spreadsheetDF.at[name, 'activated'] = spreadsheetDF.at[name, 'activated'] + ', Egg Stealer'
    spreadsheetDF.at[name, 'passive'] = spreadsheetDF.at[name, 'passive'].replace('Egg Stealer', '').replace(', , ', ', ')
    if spreadsheetDF.at[name, 'passive'].startswith(','):
        spreadsheetDF.at[name, 'passive'] = spreadsheetDF.at[name, 'passive'][1:].strip()
    elif spreadsheetDF.at[name, 'passive'].strip().endswith(','):
        spreadsheetDF.at[name, 'passive'] = spreadsheetDF.at[name, 'passive'].strip()[:-1]

#Remake passiveDF & activatedDF
def split(string) -> list[str]:
    if not isinstance(string, str):
        return []
    list1 = string.split(',')
    for string1 in list1:
        string1 = string1.strip()
    return list1
passives = spreadsheetDF['passive'].apply(split)
activateds = spreadsheetDF['activated'].apply(split)
tempDF = pd.merge(spreadsheetDF['name'], passives, left_index=True, right_index=True)
passiveDF = tempDF.explode('passive', ignore_index=True)
tempDF = pd.merge(spreadsheetDF['name'], activateds, left_index=True, right_index=True)
activatedDF = tempDF.explode('activated', ignore_index=True)
for col in passiveDF.columns:
    passiveDF[col] = passiveDF[col].str.strip()
for col in activatedDF.columns:
    activatedDF[col] = activatedDF[col].str.strip()

#Decimal formatting fix
def passiveDecimalFormattingFix(string):
    if isinstance(string, str) and 'Attack' not in string and 'Defensive' not in string:
        if '(0.' in string and string.index(')') - string.index('(') != 5:
            return string.replace(')','0)')
        if '(-0.' in string and string.index(')') - string.index('(') != 6:
            return string.replace(')','0)')
        if '1.0)' in string:
            return string.replace(')','0)')
    return string
def activatedDecimalFormattingFix(string):
    if isinstance(string, str) and 'Life Leech' in string:
        if '(0.' in string and string.index(')') - string.index('(') != 5:
            return string.replace(')','0)')
        if '1.0)' in string:
            return string.replace(')','0)')
    return string
passiveDF['passive'] = passiveDF['passive'].apply(passiveDecimalFormattingFix)
activatedDF['activated'] = activatedDF['activated'].apply(activatedDecimalFormattingFix)

# Load

The first 3 tsv files are for importing into a database (although I haven't tested them in a database management system aside from LibreBase which did not work). The last one is for opening with Google Sheets or Microsoft Excel.

In [ ]:
df.to_csv('CreatureData.tsv', sep='\t', na_rep='-1', index=False)
passiveDF.to_csv('CreaturePassive.tsv', sep='\t', index=False)
activatedDF.to_csv('CreatureActivated.tsv', sep='\t', index=False)
spreadsheetDF.to_csv('CreatureDataSpreadsheet.tsv', sep='\t', index=False)